# Projet

## Prérequis

In [57]:

import sys
import os
import importlib
from ydata_profiling import ProfileReport
import pandas as pd

PROJECT_PATH = os.getcwd()
SRC_PATH = os.path.join(PROJECT_PATH, "src")

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

print("Chemin du projet :", PROJECT_PATH)
print("Chemin du dossier src :", SRC_PATH)

import explore_data
import preprocess

importlib.reload(explore_data)
importlib.reload(preprocess)

from explore_data import * 
from preprocess import * 

Chemin du projet : c:\Users\marct\Documents\GIF-7005\Yasmine\Projet
Chemin du dossier src : c:\Users\marct\Documents\GIF-7005\Yasmine\Projet\src


ImportError: cannot import name 'TargetDriftPreset' from 'evidently.presets' (c:\Users\marct\Programs\anaconda\envs\gif7005-env-final\Lib\site-packages\evidently\presets\__init__.py)

## Prétraitement des données

In [ ]:
process_and_save_all(PROJECT_PATH, windows=["FM12"], segments=["red"])

## Importation des données prétraitées

In [ ]:
data_train = load_processed_data(PROJECT_PATH, windows=["FM12"], segments=['red'], splits=["train"])
X_train, y_train = data_train.drop(columns=['DFlag']),data_train['DFlag']
data_test = load_processed_data(PROJECT_PATH, windows=["FM12"], segments=['red'], splits=["OOS"])
X_test, y_test = data_test.drop(columns=['DFlag']), data_test['DFlag']
oot_test = load_processed_data(PROJECT_PATH, windows=["FM12"], segments=['red'], splits=["OOT"])
X_oot_test, y_oot_test = oot_test.drop(columns=['DFlag']), oot_test['DFlag']
oou_test = load_processed_data(PROJECT_PATH, windows=["FM12"], segments=['red'], splits=["OOU"])
X_oou_test, y_oou_test = oou_test.drop(columns=['DFlag']), oou_test['DFlag']

## Exploration des données

In [ ]:
from explore_data import *

save_path = os.path.join(PROJECT_PATH, "outputs", "exploration", "rapport_FM12.html")

data_to_explore = load_processed_data(PROJECT_PATH, windows=["FM12"], segments=['red'])
summarize_data_to_html(data_to_explore, "FM12 - Rapport", save_path)

### T-SNE
Cette image a été généré à partir du notebook tsne.ipynb

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

tsne_path = os.path.join(PROJECT_PATH, "outputs", "exploration", "tsne.png")
tsne_img = mpimg.imread(tsne_path)

plt.figure(figsize=(8,6))
plt.imshow(tsne_img)
plt.axis("off")
plt.show()


### Dérive des données

In [ ]:
drift_path = os.path.join(PROJECT_PATH, "outputs", "drift")

run_drift_reports(
    data_train=data_train,
    data_oos=data_test,
    data_oot=oot_test,
    data_oou=oou_test,
    output_path=drift_path
)

## Entraînement et évaluation des modèles

In [44]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

def train_and_eval(model, model_name):
    model.fit(X_train, y_train)
    results = []
    for name, (X_eval, y_eval) in {
        'OOS': (X_test, y_test),
        'OOT': (X_oot_test, y_oot_test),
        'OOU': (X_oou_test, y_oou_test)
    }.items():
        y_proba = model.predict_proba(X_eval)[:, 1]
        gini = 2 * roc_auc_score(y_eval, y_proba) - 1
        pr_auc = average_precision_score(y_eval, y_proba)
        results.append({
            'Model': model_name,
            'Dataset': name,
            'Gini': gini,
            'PR-AUC': pr_auc
        })
    return results

all_results = []

### Régression logistique

In [45]:
log_reg = LogisticRegression(
    C=0.5, penalty="l2", class_weight="balanced",
    max_iter=3000, random_state=42
)

all_results.extend(train_and_eval(log_reg, "LogisticRegression"))

### Random Forest

In [46]:
from imblearn.ensemble import BalancedRandomForestClassifier

rf_model = BalancedRandomForestClassifier(
    n_estimators=100,
    sampling_strategy="all",
    replacement=True,
    max_depth=12,
    class_weight="balanced_subsample",
    random_state=42,
    n_jobs=-1,
    bootstrap=True
)

all_results.extend(train_and_eval(rf_model, "BalancedRandomForest"))


### XGBoost

In [47]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    tree_method='hist'
)

all_results.extend(train_and_eval(xgb_model, "XGBoost"))

## Résultats

In [ ]:
import pandas as pd

# DataFrame des résultats
results_df = pd.DataFrame(all_results).drop_duplicates()

# Pivot pour avoir chaque métrique par dataset
pivot_df = results_df.pivot(index="Model", columns="Dataset", values=["Gini", "PR-AUC"])
pivot_df.columns = [f"{metric}_{ds}" for metric, ds in pivot_df.columns]
pivot_df = pivot_df.reset_index()

# DataFrame pour robustesse (écarts entre OOS et les autres datasets)
robustness = pivot_df.copy()  # On conserve toutes les performances

# Gini : écarts OOS vs OOT/OOU
robustness["Gini_Drop_OOT"] = pivot_df["Gini_OOS"] - pivot_df["Gini_OOT"]
robustness["Gini_Drop_OOU"] = pivot_df["Gini_OOS"] - pivot_df["Gini_OOU"]

# PR-AUC : écarts OOS vs OOT/OOU
robustness["PR_Drop_OOT"] = pivot_df["PR-AUC_OOS"] - pivot_df["PR-AUC_OOT"]
robustness["PR_Drop_OOU"] = pivot_df["PR-AUC_OOS"] - pivot_df["PR-AUC_OOU"]

print("\n=== Performances des modèles par jeu de données ===\n")
print(pivot_df)

print("\n=== Écarts entre OOS et les autres jeux de données ===\n")
print(robustness[["Model", "Gini_Drop_OOT", "Gini_Drop_OOU", "PR_Drop_OOT", "PR_Drop_OOU"]])



=== Écarts entre OOS et les autres jeux de données ===

                  Model  Gini_Drop_OOT  Gini_Drop_OOU  PR_Drop_OOT  \
0  BalancedRandomForest       0.016188       0.162112    -0.001959   
1    LogisticRegression       0.009230       0.164003    -0.002962   
2               XGBoost       0.086952       0.168562     0.002236   

   PR_Drop_OOU  
0    -0.046029  
1    -0.048249  
2    -0.045112  
